# 식신 - 양재역으로 검색
- 명칭, 주요 메뉴, 평점, 전화번호, 주소

In [12]:
import requests
import pandas as pd
from urllib.parse import quote

In [14]:
base_url = 'https://www.siksinhot.com'
url = f'{base_url}/search?keywords={quote("양재역")}'
req = requests.get(url)
html = req.text
#html

In [15]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

## 1. 찾으려고 하는 데이터의 태그 찾기

In [16]:
lis = soup.select('div.listTy1 > ul > li')
len(lis)

12

## 2. 여러 개의 데이터 중 하나를 선택해서 원하는 정보 추출

In [17]:
li = lis[0]
href = li.select_one('a')['href']
href

'/P/358859'

In [18]:
url = base_url + href
req = requests.get(url)
sub_soup = BeautifulSoup(req.text, 'html.parser')

In [19]:
store = sub_soup.select_one('.title')
info = store.find('h3').get_text()
info

'그릭슈바인 양재역점3.6HOT'

In [20]:
score = store.select_one('h3 > strong').string
score

'3.6'

In [21]:
info.find(score)

10

In [22]:
name = info[:info.find(score)]
name

'그릭슈바인 양재역점'

In [23]:
menu = sub_soup.select('.store_info p')[1].get_text()
menu

'나이트라이프 > 맥주/호프'

In [24]:
tel = sub_soup.select_one('.p_tel p').get_text()
tel

'02-572-6100'

In [25]:
addr = sub_soup.select_one('.txt_adr').string
addr

'서울특별시 서초구 강남대로 224'

## 3. 한 페이지에 있는 모든 데이터를 반복문으로 가져오기

In [27]:
name_list, score_list, menu_list, tel_list, addr_list = [],[],[],[],[]
for i, li in enumerate(lis):
    href = li.select_one('a')['href']
    url = base_url + href
    req = requests.get(url)
    sub_soup = BeautifulSoup(req.text, 'html.parser')

    try:
        store = sub_soup.select_one('.title')
        info = store.find('h3').get_text()
        score = store.select_one('h3 > strong').string
        name = info[:info.find(score)]
        menu = sub_soup.select('.store_info p')[1].get_text()
        tel = sub_soup.select_one('.p_tel p').get_text()
        addr = sub_soup.select_one('.txt_adr').string

        name_list.append(name)
        score_list.append(float(score))
        menu_list.append(menu)
        tel_list.append(tel)
        addr_list.append(addr)
    except:
        print(i)

3
4
